In [2]:
%%bash
pip install --upgrade pip
pip install smart_open minecart
pip install textract-trp

  Using cached pip-21.0-py3-none-any.whl (1.5 MB)


In [3]:
import time 
import re
import os
import trp
import boto3
import minecart
import logging 

import numpy as np
import pandas as pd

from smart_open import open
from sagemaker.session import Session

In [4]:
# initiate s3 bucket and corresponding data folder
bucket = "ran-s3-systemic-risk"
data_folder ="Input/X-17A-5-Subsets/"

# script to perform OCR (using Textract) for X-17A-5 subsets
out_folder = 'Output/X-17A-5-BS/'

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
s3 = boto3.client('s3')
session = Session()

# AWS Asynchronous Textract Script (requesting Job)
**Content modified from Amazon AWS Textract repository (refer to [URL](https://github.com/aws-samples/amazon-textract-code-samples/blob/master/python/12-pdf-text.py) below)** 

In [5]:
def startJob(s3BucketName:str, objectName:str) -> str:
    """
    Starts a Textract job on AWS server 
    """
    response = None
    client = boto3.client('textract')
    
    # issue response to AWS to start Textract job for table analysis 
    response = client.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3BucketName,
                'Name': objectName
            }
        },
        FeatureTypes=['TABLES']    # selecting tables from the OCR
    )
    
    # return response job ID for service
    return response["JobId"]

In [6]:
def isJobComplete(jobId:str) -> str:
    """
    Tracks the completion status of the Textract job when qued
    """
    time.sleep(1)
    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    
    # check current status of AWS job (ask server every 5 seconds for data)
    while(status == "IN_PROGRESS"):
        time.sleep(5)                   # lag before reporting status
        response = client.get_document_analysis(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    
    return status

In [7]:
def getJobResults(jobId:str) -> list:
    """
    Returns the contents of the Textract job, after completion status met
    """
    pages = []          # initialize list object to track pages

    client = boto3.client('textract')
    response = client.get_document_analysis(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
    
    # iterate through the pages and append to response figure
    while(nextToken):
        response = client.get_document_analysis(JobId=jobId, NextToken=nextToken)
        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']

    return pages

In [8]:
def runJob(bucket:str, key:str):
    """
    Function designed to call an AWS Textract 
    """
    # S3 storage for files on AWS site   
    jobId = startJob(bucket, key)   # intialize Textract job 
    print("Started job with id: {}".format(jobId))

    # if job is complete from AWS return response object 
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
        
    return response

# OCR Wrapper Functions
**The scripts perform an OCR job from AWS Textract, converting tabular data into dataframes**

In [9]:
def trp2df(table:trp.Table) -> pd.DataFrame:
    """
    Function designed to convert a trp table into a dataframe
    :param table: a trp table object parsed from a pdf  
    :return: a DataFrame object housing a textracted trp table
    
    Complexity -> O(n^2) approx.
    """
    N = len(table.rows)               # number of rows in table
    M = len(table.rows[0].cells)      # number of columns in table
    arr = [0]*N
    
    # iterate through each row within the provided table
    for row in np.arange(N):
        
        # strip the text from the cell references to construct (N X M) matrix
        arr[row] = [table.rows[row].cells[col].text.strip() for col in np.arange(M)]
        
    return pd.DataFrame(arr)

In [19]:
def readPDF(response:list) -> pd.DataFrame:
    """
    Function to transform AWS Textract object to a pdf
    :param response: AWS Textract response object
    """
    # in the event multiple tables detected on one page (concat them)
    catDF = []
    
    # format the Textract response type 
    doc = trp.Document(response)
    
    # iterate through document pages
    for page in doc.pages:
        
        # itterate through page tables
        for table in page.tables: 
            
            # convert trp-table into dataframe object
            df = trp2df(table)
            
            # remove columns that are completly empty
            empty_cols = [col for col in df.columns if (df[col] == '').all()]
            df = df.drop(empty_cols, axis=1)
  
            # number of columns in dataframe
            n = df.columns.size
            
            ##############################################################
            #                           NOTES
            #          a good dataframe should have 2-3 columns
            #      anything more or less is a reading error we ignore
            ##############################################################
            
            # if the dataframe has more than 3 columns then we most likley have an issue in parsing
            if n > 3:
                return None
            
            elif n > 1:
                
                ##############################
                # Balance Sheet Assummptions
                ##############################
                
                lineIndex = df.columns[0]

                # check for the word "cash" in a string at the begining, ignoring case sensitivity (asset check)
                assetCheck = df[lineIndex].str.contains('^Cash', regex=True, flags=re.IGNORECASE)

                # check for the word "Liabilities" in a string at the end, ignoring case sensitivity (liability check)
                debtCheck1 = df[lineIndex].str.contains('Liabilities$|^Liabilities', regex=True, flags=re.IGNORECASE)
                debtCheck2 = df[lineIndex].str.contains('Liability$|^Liability', regex=True, flags=re.IGNORECASE)
                
                # check for the presence of $ sign, we assume the balance sheet items should have presence of $ sign
                dollarCheck = df[df.columns[1]].str.contains('\$[^\]]+', regex=True, flags=re.IGNORECASE)
                
                ##############################
                # Balance Sheet Determination
                ##############################
                
                # check if the key words have been found 
                check1 = df[assetCheck | debtCheck1 | debtCheck2].empty      # check for terms, and $ presence
                check2 = df[dollarCheck == True].empty                       # check for presence of '$' sign  
                check3 = df[debtCheck1 == True].empty                        # debt check for Liabilities
                check4 = df[debtCheck2 == True].empty                        # debt check for Liability 
                
                # if either asset term or liability term is found, with a $ sign we append the dataframe
                if not check1 and not check2:
                    catDF.append(df)      # we append since sometimes asset and liablility tables are seperated 

                    if not check3 or not check4:
                        # if liability table was found on the first iteration we simply concat data frames and return 
                        return pd.concat(catDF)
        

## Extract Balance Sheet information

In [20]:
def textractParse(path:str, index:int, csvDirectory:np.ndarray, bucket:str = "ran-s3-systemic-risk", 
                  out_folder:str = 'Output/X-17A-5-BS/') -> dict:
    """
    Function runs a Textract job and saves Balance Sheet information to .csv file in s3 folder 
    """
    errors = ''
    
    # baseFile name to name export .csv file e.g. 782124-2002.csv
    baseFile = '-'.join(path.split('/')[-1].split('-')[:2])
    fileName = baseFile + '.csv'
    print('\nPerforming OCR for {}'.format(baseFile))

    # if file is not found in directory we continue the iteration process
    if out_folder + fileName not in csvDirectory:

        # temporary data frame object for balance sheet information
        res = runJob("ran-s3-systemic-risk", path)
        
        # if Textract job did not fail we continue extraction
        if res[0]['JobStatus'] != 'FAILED':
            tempDF = readPDF(res)
            print(tempDF)
            
            # checks for type of return, if none then we log an error
            if type(tempDF) == pd.DataFrame:
                
                # writing data frame to .csv file
                tempDF.to_csv(fileName, index=False)

                # save contents to AWS S3 bucket
                with open(fileName, 'rb') as data:
                    s3.put_object(Bucket=bucket, Key=out_folder + fileName, Body=data)

                # remove local file after it has been created
                os.remove(fileName)
                
                print('-----------------------------------------------------')
                print('Saved {} file to s3 bucket'.format(baseFile + '.csv'))
            else:
                errors = 'No Balance Sheet found, or parsing error'
        else:
            errors = 'Could not parse, JOB FAILEDs'
    else:
        print('{} has been downloaded'.format(fileName))
        
    return errors

In [21]:
# csv Directory to store balance sheet information 
csvs = np.array(session.list_s3_files(bucket, out_folder))

# discover all of the pdfs that you want to parse
paths = np.array(session.list_s3_files(bucket, data_folder))[1:]

errorDict = {}

# iterate through X-17A-5 subsets stored in s3 
for i, key in enumerate(paths):     
    val = textractParse(key, i, csvs)
    
    if val != '':
        errorDict[key] = val


Performing OCR for 1224385-2004
1224385-2004.csv has been downloaded

Performing OCR for 1224385-2005
1224385-2005.csv has been downloaded

Performing OCR for 1224385-2006
1224385-2006.csv has been downloaded

Performing OCR for 1224385-2007
1224385-2007.csv has been downloaded

Performing OCR for 1224385-2008
1224385-2008.csv has been downloaded

Performing OCR for 1224385-2009
1224385-2009.csv has been downloaded

Performing OCR for 1224385-2010
1224385-2010.csv has been downloaded

Performing OCR for 1224385-2011
1224385-2011.csv has been downloaded

Performing OCR for 1224385-2012
1224385-2012.csv has been downloaded

Performing OCR for 1224385-2013
1224385-2013.csv has been downloaded

Performing OCR for 1224385-2014
1224385-2014.csv has been downloaded

Performing OCR for 1224385-2015
1224385-2015.csv has been downloaded

Performing OCR for 1224385-2016
1224385-2016.csv has been downloaded

Performing OCR for 1224385-2017
1224385-2017.csv has been downloaded

Performing OCR for 

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
None

Performing OCR for 58056-2002
58056-2002.csv has been downloaded

Performing OCR for 58056-2003
58056-2003.csv has been downloaded

Performing OCR for 58056-2004
58056-2004.csv has been downloaded

Performing OCR for 58056-2005
58056-2005.csv has been downloaded

Performing OCR for 58056-2006
58056-2006.csv has been downloaded

Performing OCR for 58056-2007
58056-2007.csv has been downloaded

Performing OCR for 58056-2008
58056-2008.csv has been downloaded

Performing OCR for 58056-2009
58056-2009.csv has been downloaded

Performing OCR for 58056-2010
58056-2010.csv has been downloaded

Performing OCR for 58056-2012
58056-2012.csv has been downloaded

Performing OCR for 58056-2013
58056-2013.csv has been downloaded

Performing OCR for 58056-2014
58056-2014.csv has been downloaded

Performing OCR for 58056-2015
58056-2015.csv has been downloaded

Performing OCR 

Started job with id: 783cf60e9acccf2be4faadb8489c40292e184779513f6db2062fed15f7fada94
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
                                                    0              1
0                           Cash and cash equivalents    $ 2,696,823
1   Cash deposited with clearing organizations or ...      7,541,240
2   Financial instruments owned, at fair value (ap...               
3               U.S. government and agency securities     52,120,101
4              Other sovereign government obligations      8,587,365
5   Corporate and other debt ($52,094 related to c...     13,881,972
6                                  Corporate equities     12,898,992
7                                Derivative contracts     

                                                    0          1
0                                         ASSETS Cash    $ 2,185
1   Cash deposited with clearing organizations or ...     11,652
2   Financial instruments owned, at fair value (ap...     93,832
3   Securities received as collateral, at fair val...     16,441
4     Securities purchased under agreements to resell     63,981
5                                 Securities borrowed     95,421
6                                        Receivables:           
7                                           Customers     11,766
8         Brokers, dealers and clearing organizations      7,078
9                              Interest and dividends        573
10                                     Fees and other        450
11                                         Affiliates         30
12                                       Other assets        397
13                                       Total assets  $ 303,806
14                    LIA

Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
                                                    0             1  \
0                                              Assets                 
1                           Cash and cash equivalents                 
2   Cash and securities segregated pursuant to fed...                 
3     Collateralized short-term financing agreements:                 
4     Securities purchased under agreements to resell  $ 25,660,248   
5                                 Securities borrowed    18,682,905   
6   Financial instruments owned (a

...

Started job with id: 9f1ae5b86fa05e7c459aab3b4130e31eb53d1af90af70c2e9d3b6eda4c1dfcf5
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
                                                    0         1          2
0                                              Assets                     
1                           Cash and cash equivalents                $ 730
2   Cash segregated and on deposit for Federal and...                     
3            or deposited with clearing organizations               14,085
4     Collateralized short-term financin

Started job with id: 7a4462a08697e52b785e4d8d1f76c780b5e1cb79d64feadf562a10d40cec2370
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
Resultset page recieved: 8
                                                    0         1           2
0                                              Assets                      
1                           Cash and cash equivalents                 $ 699
2   Cash segregated and on deposit for Federal and...                10,691
3     Collateralized short-term financing agreements:                      
4   Secu

Started job with id: ae9a4fac1ad1c20b0023e996712204c37a63cc8eda73a00fb5dc44e22067148a
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
None

Performing OCR for 91154-2016
Started job with id: a826da612fd854e8859414b38466069a8a4b373ccfd3962cc999d0fbaa55bdd3
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
                                                    0          1
0                                              Assets           


Started job with id: 755f11d0c91bbae07cac0a09cb95c32206d08604fece8ee7aabb42ab538bfc14
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
                                                    0          1
0                                              Assets           
1                           Cash and cash equivalents      $ 471
2   Cash segregated under federal and other regula...      4,675
3   Securities borrowed or purchased under agreeme...    162,683
4   Trading account assets ($23,829 pledged as col...           
5                          Mortgage-backed securities     24,642
6         U.S. Treasury and federal agency securities     14,351
7                                   Equity securities      7,52

Job status: SUCCEEDED
Resultset page recieved: 1
Resultset page recieved: 2
Resultset page recieved: 3
Resultset page recieved: 4
Resultset page recieved: 5
Resultset page recieved: 6
Resultset page recieved: 7
Resultset page recieved: 8
                                                    0          1
0                                              Assets           
1                                                Cash      $ 798
2   Cash segregated under federal and other regula...      6,055
3   Securities borrowed and purchased under agreem...    171,131
4   Trading account assets ($26,266 pledged as col...           
5                          Mortgage-backed securities     29,560
6         U.S. Treasury and federal agency securities     18,175
7                                   Equity securities     12,389
8                           Corporate debt securities      6,806
9                             Asset-backed securities      2,435
10                     State and municipal secu

In [ ]:
errorDict